In [1]:
wkt_footprint = "POLYGON((-55.16629529644229 -34.7494869239046,-55.02038312603214 -34.7494869239046,-55.02038312603214 -34.868725532230165,-55.16629529644229 -34.868725532230165,-55.16629529644229 -34.7494869239046))"
data_path = "data"
product_path = "data/2022-02-10/S3B_SL_2_LST____20220210T132713_20220210T133013_20220211T214624_0179_062_252_3600_LN2_O_NT_004.SEN3"

# Problemas

## Algunos productos tienen su .zip corrupto y no se puede descomprimir:
S3A_SL_2_LST____20180518T125419_20180518T143518_20180519T203221_6059_031_195______LN2_O_NT_003
S3A_SL_2_LST____20180418T123152_20180418T141251_20180419T194201_6059_030_152______LN2_O_NT_003
S3A_SL_2_LST____20180523T003729_20180523T021828_20180524T063958_6059_031_259______LN2_O_NT_003
S3A_SL_2_LST____20180523T122423_20180523T140522_20180524T194914_6059_031_266______LN2_O_NT_003
S3A_SL_2_LST____20180526T124650_20180526T142749_20180527T200646_6059_031_309______LN2_O_NT_003

## Algunos products no se logran cargar correctamente despues de descomprimidos:
S3A_SL_2_LST____20180217T014056_20180217T032155_20180930T083426_6059_028_060______LR1_R_NT_003
S3A_SL_2_LST____20180309T130909_20180309T145008_20180930T091302_6059_028_352______LR1_R_NT_003
S3A_SL_2_LST____20190227T130531_20190227T144631_20190228T233400_6059_042_024______LN2_O_NT_003
S3A_SL_2_LST____20200306T115429_20200306T133528_20200307T195439_6059_055_337______LN2_O_NT_004
S3B_SL_2_LST____20200321T130712_20200321T144811_20200322T194334_6059_037_024______LN2_O_NT_004


In [2]:
import seaborn as sns
import matplotlib.pylab as plt
from datetime import datetime

from LST_Processing import LST, LSTDataset, find_best_temperature_candidate

### Get and display data from a defined date

In [ ]:
lst_dataset = LSTDataset(data_path, wkt_footprint)
temperature_data, errors = lst_dataset.get_day_data(datetime.date(datetime(2018,2,17)))

In [ ]:
for lst in temperature_data:
    print(lst.capture_datetime)
    print(lst.water_temperature)
    ax = sns.heatmap(lst.lst, linewidth=0.5)
    plt.show()
    
    ax = sns.heatmap(lst.valid_pixels_temp, linewidth=0.5)
    plt.show()
    print("/////////////////////////")

### Get water temperature from every Sentinel-2 valid day

In [3]:
import settings
from process_sentinel2 import DayDataGenerator

DATA_PATH = settings.data_path
DATE_FORMAT = '%Y-%m-%d'
START_DATE = '2016-12-21'
END_DATE = '2022-04-20'

data_generator = DayDataGenerator(START_DATE, END_DATE, DATE_FORMAT, DATA_PATH, skip_invalid=True)

valid_dates = sorted([day.date.date() for day in data_generator])
print("Valid Sentinel-2 days:", len(valid_dates))

Valid Sentinel-2 days: 108


In [4]:
import sys
import subprocess

for vd in valid_dates:
    print(vd)
    result = subprocess.run([sys.executable, "LST_Processing.py", "-i", str(vd)], stdout=subprocess.PIPE)

2017-11-19
2017-11-29
2017-12-12
2017-12-29
2018-01-08
2018-01-11
2018-01-21
2018-01-31
2018-02-07
2018-02-15
2018-02-17
2018-03-02
2018-03-07
2018-03-09
2018-03-14
2018-03-17
2018-03-22
2018-04-18
2018-05-18
2018-05-23
2018-05-26
2018-11-04
2018-11-09
2018-11-22
2018-12-12
2018-12-24
2019-01-01
2019-01-08
2019-01-11
2019-01-28
2019-02-02
2019-02-05
2019-02-10
2019-02-17
2019-02-27
2019-03-07
2019-05-01
2019-11-07
2019-11-27
2019-11-29
2019-12-14
2020-01-08
2020-01-11
2020-01-26
2020-01-28
2020-01-31
2020-02-27
2020-03-01
2020-03-03
2020-03-06
2020-03-08
2020-03-13
2020-03-21
2020-03-23
2020-04-05
2020-04-20
2020-05-10
2020-05-12
2020-05-17
2020-05-20
2020-05-27
2020-05-30
2020-11-01
2020-11-23
2020-11-28
2020-12-01
2020-12-16
2020-12-23
2021-01-02
2021-01-10
2021-01-20
2021-01-22
2021-01-25
2021-02-06
2021-02-14
2021-02-19
2021-02-24
2021-02-26
2021-03-28
2021-04-07
2021-04-12
2021-04-27
2021-04-30
2021-05-12
2021-05-15
2021-05-25
2021-05-27
2021-05-30
2021-11-08
2021-11-18
2021-12-26

In [5]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from datetime import datetime

with open("prediction_data/temp_output_all-valid-captures.txt") as file:
    temps = file.readlines()

temp_data = [{"timestamp":datetime.strptime(d.split(",")[0],'%Y-%m-%d %H:%M'), "temperature":float(d.split(",")[1])} for d in temps if "." in d.split(",")[1]]
temp_df = pd.DataFrame(temp_data)

temp_df = temp_df.set_index("timestamp")
#temp_df = temp_df.between_time('00:00', '03:00')
temp_df = temp_df.between_time('11:00', '15:00')
print(temp_df)

                     temperature
timestamp                       
2017-11-19 12:20:00    17.317070
2017-11-29 13:01:00    21.506166
2017-12-12 12:24:00    24.530767
2018-01-08 12:24:00    29.250406
2018-01-11 12:46:00    29.555037
...                          ...
2022-03-11 13:14:00    23.794353
2022-03-16 12:44:00    23.757390
2022-04-02 13:04:00    18.746542
2022-04-10 12:57:00    21.511007
2022-04-10 13:36:00    22.085226

[131 rows x 1 columns]


In [6]:
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook, push_notebook, show
from bokeh.transform import dodge
output_notebook()

source_temp = ColumnDataSource(temp_df)

hover_temp = HoverTool(names = ['Temp/timestamp'],
                  tooltips=[('Temperature (C)','@temperature'), ("Capture time", "@timestamp{%F}")],
                    formatters={'@timestamp': 'datetime'},
                  )
#output_file(filename="RAM_and_GPU_memory.html", title="Static HTML file")
p = figure(x_axis_type='datetime', x_axis_label='timestamp', y_axis_label='Water average temperature', width=800, height=300, name="scatter")
p.add_tools(hover_temp)
p.line(x='timestamp', y='temperature', source=source_temp, hover_alpha=1, name="Temp/timestamp")
handle = show(p, notebook_handle=True)
push_notebook(handle=handle)
#save(p)

Loading BokehJS ...